<a href="https://colab.research.google.com/github/sunlaito/LearnPennyLane/blob/main/Basic3_Hybrid_computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ref: https://pennylane.ai/qml/demos/tutorial_plugins_hybrid.html

In [ ]:
# numba>=0.49.1 as required by pennylane-sf
!pip install numba --upgrade

In [ ]:
# urllib3>=1.21.1, !=1.25.0, !=1.25.1, <1.26 as required by pennylane-sf
!pip install urllib3==1.25.11

In [ ]:
!pip install pennylane --upgrade

In [ ]:
!pip install pennylane-sf

In [ ]:
import pennylane as qml
from pennylane import numpy as np
qml.about()
# Note that the devices from pennylane-sf are not recognized

Name: PennyLane
Version: 0.13.0
Summary: PennyLane is a Python quantum machine learning library by Xanadu Inc.
Home-page: https://github.com/XanaduAI/pennylane
Author: None
Author-email: None
License: Apache License 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: semantic-version, scipy, appdirs, toml, numpy, networkx, autograd
Required-by: PennyLane-SF
Platform info:           Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
Python version:          3.6.9
Numpy version:           1.19.5
Scipy version:           1.4.1
Installed devices:


In [ ]:
# Runtime -> Restart runtime when running in Google Colab, otherwise the devices are not recognized
import pennylane as qml
from pennylane import numpy as np
qml.about()

Name: PennyLane
Version: 0.13.0
Summary: PennyLane is a Python quantum machine learning library by Xanadu Inc.
Home-page: https://github.com/XanaduAI/pennylane
Author: None
Author-email: None
License: Apache License 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: semantic-version, networkx, scipy, autograd, numpy, appdirs, toml
Required-by: PennyLane-SF
Platform info:           Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
Python version:          3.6.9
Numpy version:           1.19.5
Scipy version:           1.4.1
Installed devices:
- default.gaussian (PennyLane-0.13.0)
- default.mixed (PennyLane-0.13.0)
- default.qubit (PennyLane-0.13.0)
- default.qubit.autograd (PennyLane-0.13.0)
- default.qubit.tf (PennyLane-0.13.0)
- default.tensor (PennyLane-0.13.0)
- default.tensor.tf (PennyLane-0.13.0)
- strawberryfields.fock (PennyLane-SF-0.12.0)
- strawberryfields.gaussian (PennyLane-SF-0.12.0)
- strawberryfields.gbs (PennyLane-SF-0.12.0)
- strawberryfields.remote (PennyLane-

In [ ]:
# If numba and urllib3 with the right versions are not installed before, this step will go wrong

# As soon as the PennyLane-SF plugin is installed, 
# the 'strawberryfields.fock' device can be loaded — no additional commands or library imports required.
# cutoff_dim = D: The simulator represents quantum states in the Fock basis |0>,...,|D-1>.
dev_fock = qml.device("strawberryfields.fock", wires=2, cutoff_dim=2)

dev_qubit = qml.device("default.qubit", wires=1)

In [ ]:
# Hybrid computation

In [ ]:
# QNode
@qml.qnode(dev_qubit)
def qubit_rotation(phi1, phi2):
    """Qubit rotation QNode"""
    qml.RX(phi1, wires=0)
    qml.RY(phi2, wires=0)
    return qml.expval(qml.PauliZ(0))


@qml.qnode(dev_fock)
def photon_redirection(params):
    """The photon redirection QNode"""
    qml.FockState(1, wires=0)
    qml.Beamsplitter(params[0], params[1], wires=[0, 1])
    return qml.expval(qml.NumberOperator(1))

def squared_difference(x, y):
    """Classical node to compute the squared
    difference between two inputs"""
    return np.abs(x - y) ** 2

In [ ]:
# Optimization
def cost(params):
    return -photon_redirection(params)

In [ ]:
init_params = np.array([0.01, 0.01])
print(cost(init_params))

-9.999666671111081e-05


In [ ]:
# Note that phi1 and phi2 are fixed. We only  optimize params in the following codes. 
def cost(params, phi1=0.5, phi2=0.1):
    """Returns the squared difference between
    the photon-redirection and qubit-rotation QNodes, for
    fixed values of the qubit rotation angles phi1 and phi2"""
    qubit_result = qubit_rotation(phi1, phi2)
    photon_result = photon_redirection(params)
    return squared_difference(qubit_result, photon_result)

In [ ]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 100
# set the initial parameter values
params = np.array([0.01, 0.01])

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.2154539
Cost after step    10:  0.0000982
Cost after step    15:  0.0000011
Cost after step    20:  0.0000000
Cost after step    25:  0.0000000
Cost after step    30:  0.0000000
Cost after step    35:  0.0000000
Cost after step    40:  0.0000000
Cost after step    45:  0.0000000
Cost after step    50:  0.0000000
Cost after step    55:  0.0000000
Cost after step    60:  0.0000000
Cost after step    65:  0.0000000
Cost after step    70:  0.0000000
Cost after step    75:  0.0000000
Cost after step    80:  0.0000000
Cost after step    85:  0.0000000
Cost after step    90:  0.0000000
Cost after step    95:  0.0000000
Cost after step   100:  0.0000000
Optimized rotation angles: [1.20671364 0.01      ]


In [ ]:
result = params # [1.20671364, 0.01]
print(photon_redirection(result))
print(qubit_rotation(0.5, 0.1))

0.8731983044562817
0.8731983044562817
